### connect mongoDB

In [6]:
import re
import codecs
from pymongo import MongoClient 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import jieba
import time
#coding=utf-8
import sys
import json
jieba.set_dictionary('E:dics/dict.txt.big.txt')   #切換至中文繁體字庫
jieba.load_userdict('E:dics/dict_keyw.txt')       #加入自建詞庫
jieba.load_userdict('E:dics/dict_keyw_new.txt')


client = MongoClient('10.120.28.6',27017)
database = client['test_new1']
collection =database['test1']

def tfIdf(content):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(content)  
    weight = X.toarray()
    features = vectorizer.get_feature_names()
    print "特徵值數量:",len(features)
    return weight, features


def getTopWeight(weight, features, resp_count, top_n):
    top_features = []
    for n in range(0,resp_count):  #迴圈參考上面的總文章數
        indices = np.argsort(weight[n])[::-1]  
        # transformer = TfidfTransformer()  #X.toarray()[5] 是第幾篇新聞的意思
        
        # 看TOP多少的詞
        #top_n = 30
        top_features.append([features[i] for i in indices[:top_n]]) #這邊的寫法會讓關鍵字中間會有空白
    return top_features


Building prefix dict from E:dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u93a38cfbc8cc055f8a9347c363a92cd4.cache
Loading model cost 0.562 seconds.
Prefix dict has been built succesfully.


### 存檔案

In [21]:
tag1 = "_"
#tag2 = "颱風"

querry_resp = collection.find({"$and":[
            {"content":{"$regex":tag1}},
            #{"content":{"$regex":tag2}},
        ]})

#querry_resp = collection.find({
#        "date":{"$regex":"201601"}
#    })
print querry_resp.count()

with codecs.open("C:/Users/BIG DATA/pythonETL/project/test/project_TFIDF.txt",'w',encoding='utf-8') as f:
    for post in querry_resp: 
        writer=[]
        #writer =  str(post['_id']) + "|"+post['content']
        writer = post['content']
        #print  type(summary)
        #writer=('/'.join(jieba.cut(summary)))  #把一篇文章切完詞後 再用 "/" 接起來放在一起!
        writer1=writer+'\n'
        f.write(writer1) 

6217


### 讀檔+TFIDF

In [27]:
#讀檔案
time_start = time.clock()
with codecs.open("C:/Users/BIG DATA/pythonETL/project/test/project_TFIDF.txt",'r',encoding='utf-8') as f:
    contents_IO=f.readlines()
    #print len(contents_IO)
    #for line in contents:
        #print line

content=[]
for post in contents: 
    summary = post
    content.append('/'.join(jieba.cut(summary)))
    #title.append(post['title'])
    
#總文章數量
resp_count = len(contents)
print "查詢結果資料總筆數:", resp_count
time_step1 = time.clock()
print " I/O time 的時間", time_step1-time_start, "秒"

weight, features = tfIdf(content)
time_step2 = time.clock()
print "計算tf-idf的時間", time_step2-time_step1, "秒"

#resp_num #querr回來的第幾篇
top_num = 20   #取tf-idf前幾高的
tf_idf_resp = getTopWeight(weight, features, resp_count, top_num)
time_step3 = time.clock()
print "排序與取值的時間", time_step3-time_step2, "秒"

查詢結果資料總筆數: 6217
 I/O time 的時間 12.9228714492 秒
特徵值數量: 117955
計算tf-idf的時間 2.22021583844 秒
排序與取值的時間 11.0162351947 秒
